In [94]:
import subprocess
from pathlib import Path
import pandas as pd
from collections import Counter as count
import pydicom
import re
import io
import shutil
import dicom2nifti
import dicom2nifti.settings as settings
import nibabel as nib
import numpy as np

In [20]:
zips_path = Path('ullevaal_zips/')
dicom_path = Path('klinisk_baseline_ulleval_may2022/')
axial_ct_path = Path('ullevaal_axial_cts/')
missing_cts_path = Path('missing_cts/')
axial_missing_ct_series = Path('ullevaal_missing_cts/axial_ct_dicom')
buffer = Path('buffer/')

In [3]:
ct_scans_99 = pd.read_table('99_ct_scans.txt')
ct_scans_90 = pd.read_table('90_ct_scans.txt')

In [4]:
ct_scans_99[['Patient name', 'Comments']]
ct_scans_90[['Patient name', 'Comments']]
a = set(ct_scans_99['Comments'])
b = set(ct_scans_90['Comments'])
missing_suids = a - b
missing_cts = pd.DataFrame({'StudyInstanceUID': list(missing_suids)})

In [5]:
missing_cts.to_csv('suid2.txt', sep=' ', index=False)

In [6]:
suid_input = 'suid2.txt'

In [7]:
zips = [z for z in zips_path.glob('*.zip')]

In [8]:
input_dicom_file_paths = [p for p in dicom_path.glob('**/*') if p.is_file()]

In [9]:
tag_search_term = re.compile('ax', re.IGNORECASE)

In [16]:
def filter_dicoms(path_to_dcm_file, pattern):
    dcm = pydicom.dcmread(path_to_dcm_file, stop_before_pixels=True, force=True)
    if isinstance(dcm, pydicom.dicomdir.DicomDir):
        pass
    elif isinstance(dcm, pydicom.dataset.FileDataset):
        if hasattr(dcm, 'Modality'): 
            if dcm.Modality == 'CT':
                if hasattr(dcm, 'ContrastBolusRoute'):
                    return
                if hasattr(dcm, 'ContrastBolusVolume'):
                    return
                if hasattr(dcm, 'ImageType'):
                    if dcm.ImageType[2] == 'LOCALIZER':
                        return 
                if hasattr(dcm, 'SeriesDescription'):
                    if pattern.search(dcm.SeriesDescription):
                        series_ids.append(dcm.SeriesInstanceUID)
                        shutil.copy(path_to_dcm_file, buffer)

In [18]:
series_ids = []
for p in input_dicom_file_paths:
    filter_dicoms(p, tag_search_term)

In [13]:
series_uids = set(series_ids)
len(series_uids)

156

In [14]:
series_uids_list = list(series_uids)
series_uids_list.insert(0, 'SeriesInstanceUID')
string_input = '\n'.join(series_uids_list)
suid_txt = open('suid.txt', 'w')
suid_txt.write(string_input)
suid_txt.close()

In [65]:
#for zip in zips:
#    subprocess.call(['unzip', zip.as_posix(), '-d', dicom_path.as_posix()])

In [15]:
dicompull_location = '/Applications/dicomtools-0.8.13-mac/dicompull'

In [58]:
def run_dicompull(tag):
    subprocess.call([dicompull_location, '-k', 'Modality=CT', '-k', '0008,103e='+tag, dicom_path.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT'])

In [18]:
def run_dicompull_suid(suids):
    subprocess.call([dicompull_location, '-u', suids, dicom_path.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT'])

In [77]:
def dicompull_sort():
    subprocess.call([dicompull_location, buffer.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT']) 

In [26]:
def dicompull():
    subprocess.call([dicompull_location, buffer.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT']) 

## Manually copy missing ct scans to buffer folder

In [23]:
run_dicompull_suid('suid2.txt')

Scanning 100%                      
Copying 100%                       


In [21]:
tags = ['ax til PACS', 'ax', 'Ax til PACS', 'Axial', 'AX', 'Caput', 'ax til pacs']

In [13]:
for tag in tags:
    run_dicompull(tag)

Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      


In [36]:
bla = [p for p in axial_ct_path.glob('*ct')]

In [37]:
len(bla)

99

In [23]:
buffer_missing = buffer / 'missing'

In [24]:
shutil.copytree(axial_missing_ct_series, buffer_missing)

PosixPath('buffer/missing')

In [25]:
output_dir = axial_ct_path / 'CT'

In [29]:
def rename_directories():
    '''Rename output_dir. Only run once from console after dicompull!'''
    output_dir = axial_ct_path / 'CT'
    for p in output_dir.glob('*'):
        p.rename(p.parents[0]/ p.name[:6])

In [31]:
settings.disable_validate_orthogonal()
settings.disable_validate_slice_increment()

In [32]:
for p in output_dir.glob('*'):
    dicom_directory = p / 'dicom' / 'CT' 
    subprocess.call(['dcm2niix', '-o', p.as_posix(), '-f', p.name+'_Brain_CT', '-b', 'y', '-z', 'y',  dicom_directory.as_posix()])
    #dicom2nifti.dicom_series_to_nifti(dicom_directory, p.as_posix()+'/'+p.name+'_Brain_CT.nii.gz', reorient_nifti=False) 
    print(p.as_posix())

Chris Rorden's dcm2niiX version v1.0.20220720  Clang13.0.1 x86-64 (64-bit MacOS)
Found 54 DICOM file(s)
Convert 54 DICOM as ullevaal_axial_cts/CT/100601/100601_Brain_CT (512x512x54x1)
Conversion required 2.064583 seconds (2.017597 for core code).
ullevaal_axial_cts/CT/100601
Chris Rorden's dcm2niiX version v1.0.20220720  Clang13.0.1 x86-64 (64-bit MacOS)
Found 51 DICOM file(s)
Convert 51 DICOM as ullevaal_axial_cts/CT/100265/100265_Brain_CT (512x614x51x1)
Conversion required 2.561759 seconds (2.503352 for core code).
ullevaal_axial_cts/CT/100265
Chris Rorden's dcm2niiX version v1.0.20220720  Clang13.0.1 x86-64 (64-bit MacOS)
Found 56 DICOM file(s)
Slices not stacked: orientation varies (vNav or localizer?) [0.999336 -0.00312486 0.0362914 -0.00968627 0.937644 0.347461] != [0.0683531 0.980955 -0.18181 0 -0.182236 -0.983255]
Convert 55 DICOM as ullevaal_axial_cts/CT/100434/100434_Brain_CT_i00002 (512x512x55x1)
Convert 1 DICOM as ullevaal_axial_cts/CT/100434/100434_Brain_CT_i00001 (512x512

In [35]:
ct_nifti_path = Path.home() / 'native_ct/ullevaal_axial_cts/CT'

In [103]:
nifti_paths = [p for p in ct_nifti_path.glob('**/*.nii.gz')]

In [87]:
def get_nifti_shape(nifti):
    slices = nifti.header.get_data_shape()[2]
    return slices

In [89]:
dicom_paths = [p for p in ct_nifti_path.glob('**/*.dcm')]

In [100]:
def filter_dicoms(path_to_dcm_file):
    dcm = pydicom.dcmread(path_to_dcm_file, stop_before_pixels=True, force=True)
    if isinstance(dcm, pydicom.dicomdir.DicomDir):
        pass
    elif isinstance(dcm, pydicom.dataset.FileDataset):
        if hasattr(dcm, 'ImageOrientationPatient'):
            iop_round = [round(x) for x in dcm.ImageOrientationPatient]
            plane = np.cross(iop_round[0:3], iop_round[3:6])
            plane = [abs(x) for x in plane]
            if plane[2] != 1:
                path_to_dcm_file.unlink()
                return "Deleted non-axial slice!"
                   # if pattern.search(dcm.SeriesDescription):
                   #     series_ids.append(dcm.SeriesInstanceUID)
                   #     shutil.copy(path_to_dcm_file, buffer)

In [101]:
for p in dicom_paths:
    print(filter_dicoms(p))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Deleted non-axial slice!
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [88]:
for p in nifti_paths:
    img = nib.load(p)
    if get_nifti_shape(img) == 1:
            stem = p.name.removesuffix('.nii.gz')
            for suffix in ['.nii.gz','.json']:
                stem = p.name.removesuffix('.nii.gz')
                filename = str(stem) + suffix
                path_to_delete = p.parent / filename
                path_to_delete.unlink()
                print("deleted: " + path_to_delete.as_posix())

deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100501/100501_Brain_CT_i00001.nii.gz
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100501/100501_Brain_CT_i00001.json
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100795/100795_Brain_CT_i00001.nii.gz
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100795/100795_Brain_CT_i00001.json
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100761/100761_Brain_CT_i00001.nii.gz
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100761/100761_Brain_CT_i00001.json
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100598/100598_Brain_CT_i00001.nii.gz
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100598/100598_Brain_CT_i00001.json
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100351/100351_Brain_CT_i00001.nii.gz
deleted: /Users/tillschellhorn/native_ct/ullevaal_axial_cts/CT/100351/100351_Brain_CT_i00001.json
deleted: /

In [102]:
!tar -cjvf ullevaal_300822.tar.bz2 {output_dir.as_posix()}

a ullevaal_axial_cts/CT
a ullevaal_axial_cts/CT/100601
a ullevaal_axial_cts/CT/100265
a ullevaal_axial_cts/CT/100434
a ullevaal_axial_cts/CT/100298
a ullevaal_axial_cts/CT/100836
a ullevaal_axial_cts/CT/100809
a ullevaal_axial_cts/CT/100697
a ullevaal_axial_cts/CT/100469
a ullevaal_axial_cts/CT/100467
a ullevaal_axial_cts/CT/100403
a ullevaal_axial_cts/CT/100607
a ullevaal_axial_cts/CT/100432
a ullevaal_axial_cts/CT/100830
a ullevaal_axial_cts/CT/100837
a ullevaal_axial_cts/CT/100808
a ullevaal_axial_cts/CT/100522
a ullevaal_axial_cts/CT/100316
a ullevaal_axial_cts/CT/100744
a ullevaal_axial_cts/CT/100343
a ullevaal_axial_cts/CT/100716
a ullevaal_axial_cts/CT/100381
a ullevaal_axial_cts/CT/100789
a ullevaal_axial_cts/CT/100328
a ullevaal_axial_cts/CT/100590
a ullevaal_axial_cts/CT/100794
a ullevaal_axial_cts/CT/100303
a ullevaal_axial_cts/CT/100501
a ullevaal_axial_cts/CT/100792
a ullevaal_axial_cts/CT/100795
a ullevaal_axial_cts/CT/100761
a ullevaal_axial_cts/CT/100598
a ullevaal_axia